In [1]:
import requests, time, re, os, json, mysql.connector
#import mysql, pymysql, pyodbc
#import sqlalchemy as sal
#from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
#from bs4 import BeautifulSoup as bs
from datetime import date

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#importing credentials from txt file
with open('../../../Notes-General/config.txt', 'r') as f:
    creds = f.read()
creds = json.loads(creds)

# path to downloads folder
download_path = creds['downloads']

In [2]:
#### using the save as button on the page to save the files individually.
#### the rows of the tables that are not visible are hidden from HTML
#### saving is the only way to get all the rows that I know at this time

# path to webdriver for selenium
service = Service(r"..\browser\geckodriver.exe")

driver = webdriver.Firefox(service=service)
waitTime = 10

site = "https://www.rotowire.com/betting/nba/player-props.php"
driver.get(site)

# will be used to loop through the webpage
data_prop_names = ["PTS","REB","AST",
                   "THREES","BLK","STL",
                   "TURNOVERS","PTSREBAST","PTSREB",
                   "PTSAST","REBAST","STLBLK"]
missing = []
for i in range(len(data_prop_names)):
# find the element in the source
    try:
        ele = driver.find_element(By.CSS_SELECTOR, "div[data-prop={}]".format(data_prop_names[i]))
        # retrieve its page coords
        loc = ele.location
        #scroll to the location to activate the table creation
        driver.execute_script("window.scrollTo(0, {});".format(str(loc["y"])))
        # pause for data loading
        time.sleep(3)
        ###### save to csv 
        export_button = ele.find_element(By.CLASS_NAME, "is-csv")
        export_button.click()
    except Exception as e:
        print(data_prop_names[i], e)
        missing.append(i)
        continue
    
driver.close()

BLK Message: Unable to locate element: .is-csv
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

STL Message: Unable to locate element: .is-csv
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

TURNOVERS Message: Unable to locate element: .is-csv
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.

In [3]:
# columns to use for renaming csv's
columns = ['PLAYER', 'team', 'opp', 'dk_line', 'dk_oOdds', 'dk_uOdds',
          'fd_line', 'fd_oOdds', 'fd_uOdds','m_line', 'm_oOdds', 'm_uOdds',
          'pb_line', 'pb_oOdds', 'pb_uOdds']
# will be used to get the correct file number when there are missing files
count = 0
# loop through each file to agg into a single data frame
for i in range(0, len(data_prop_names)):
    #uses the indices for the missing odds files
    if i in missing:
        continue
    else:
        stat = data_prop_names[i]
        if i == 0:
            path = download_path + "nba-player-props-rotowire.csv"
            data = pd.read_csv(path, skiprows=1)
            data.columns=columns
            data['stat'] = stat
        else:
            count += 1
            path = download_path + "nba-player-props-rotowire({}).csv"
            temp = pd.read_csv(path.format(count), skiprows=1)
            temp.columns=columns
            temp['stat'] = stat
            # concat data into main df    
            data = pd.concat([data, temp])

# using dk line as the default and averaging the other options when DK is not available
data["line"] = np.where(data["dk_line"].isna(), 
                       data[["fd_line","m_line","pb_line"]].mean(axis=1, skipna=True),
                       data["dk_line"])
data["oOdds"] = np.where(data["dk_oOdds"].isna(), 
                       data[["fd_oOdds","m_oOdds","pb_oOdds"]].mean(axis=1, skipna=True),
                       data["dk_oOdds"])
data["uOdds"] = np.where(data["dk_uOdds"].isna(), 
                       data[["fd_uOdds","m_uOdds","pb_uOdds"]].mean(axis=1, skipna=True),
                       data["dk_uOdds"])

data = data[["PLAYER", "team", "stat", "line", "oOdds", "uOdds"]]
# adding the date of the odds
data.loc[:,'date'] = date.today()  

In [4]:
data[data["stat"]=="REB"]

,PLAYER,team,stat,line,oOdds,uOdds,date
0,Domantas Sabonis,SAC,REB,12.5,-105.0,-125.0,2023-02-05
1,Julius Randle,NYK,REB,11.5,100.0,-130.0,2023-02-05
2,Jarrett Allen,CLE,REB,10.5,110.0,-140.0,2023-02-05
3,Joel Embiid,PHI,REB,10.5,110.0,-140.0,2023-02-05
4,Evan Mobley,CLE,REB,9.5,105.0,-135.0,2023-02-05
5,Precious Achiuwa,TOR,REB,8.5,-105.0,-125.0,2023-02-05
6,Myles Turner,IND,REB,8.5,-105.0,-125.0,2023-02-05
7,Scottie Barnes,TOR,REB,7.5,-120.0,-110.0,2023-02-05
8,Pascal Siakam,TOR,REB,7.5,-110.0,-120.0,2023-02-05
9,James Harden,PHI,REB,5.5,-140.0,110.0,2023-02-05


In [ ]:
date_string = str(date.today())
data.to_csv("../data/{}_odds.csv".format(date_string), index=False)

In [ ]:
data

In [ ]:
### this facilitates programmatically accessing each prop table data
### but only grabs the players visible on initial load. I don't know
### how to scroll the div within the page yet.

try:
    # path to webdriver for selenium
    service = Service(r"..\browser\geckodriver.exe")

    driver = webdriver.Firefox(service=service)
    waitTime = 10

    site = "https://www.rotowire.com/betting/nba/player-props.php"
    driver.get(site)


    # will be used to loop through the webpage
    data_prop_names = ["PTS","REB","AST","THREES","BLK","STL","TURNOVERS","PTSREBAST","PTSREB","PTSAST","REBAST","STLBLK"]
    for i in data_prop_names:
    # find the element in the source
        print(i)
        ele = driver.find_element(By.CSS_SELECTOR, "div[data-prop={}]".format(i))
        # retrieve its page coords
        loc = ele.location
        #scroll to the location to activate the table creation
        driver.execute_script("window.scrollTo(0, {});".format(str(loc["y"])))
        # pause for data loading
        time.sleep(1)

        #re-search for the element to get the data with the table populated
        ele = driver.find_element(By.CSS_SELECTOR, "div[data-prop={}]".format(i))

        names = ele.find_element(By.CLASS_NAME, "webix_ss_left")
        remaining_data = ele.find_element(By.CLASS_NAME, "webix_ss_center")

        ##### names #####
        n = names.find_elements(By.CLASS_NAME, "webix_cell  ")
        PLAYER = []
        stat = []
        for j in n:
            name = " ".join(j.text.split()).strip()
            PLAYER.append(name)
            stat.append(i.lower())
            #row = i.get_attribute('aria-rowindex')
            #col = i.get_attribute('aria-colindex')

        # pulling team and book line/odds data
        cols = remaining_data.find_elements(By.CLASS_NAME, "webix_column ")

        ##### teams #####
        teams = cols[0].find_elements(By.CLASS_NAME, "webix_cell  ")
        team = []
        for t in teams:
            team.append(t.text)

        # this holds the location of each data point in remaining_data
        # loc is the index and data will hold the actual line values
        books = {
            2:{"data":[]}, 3:{"data":[]}, 4:{"data":[]},
            5:{"data":[]}, 6:{"data":[]}, 7:{"data":[]},
            8:{"data":[]}, 9:{"data":[]}, 10:{"data":[]},
            11:{"data":[]}, 12:{"data":[]}, 13:{"data":[]}
        }

        for j in books.keys():
            #HTML hides HTML source data if it is not all visible on screen, pointsBET uOdds are offscreen
            #I can't figure out FIrefox zoom with Selenium so I am skipping Pointsbet for this grouping
            try:
                temp_data = cols[j].find_elements(By.CLASS_NAME, "webix_cell  ")
                for h in temp_data:
                    try:
                        books[j]['data'].append(float(h.text))
                    except:
                        books[j]['data'].append(np.nan)
            except:
                break

        #using  PTS to start the main dataframe since each table is not sorted the same way
        if i == "PTS":
            # create main dataframe that will receive all the other data points
            data = pd.DataFrame(list(zip(PLAYER, team, stat, 
                                 books[2]['data'], books[3]['data'], books[4]['data'], 
                                 books[5]['data'], books[6]['data'], books[7]['data'],
                                 books[8]['data'], books[9]['data'], books[10]['data'],
                                 books[11]['data'], books[12]['data'], books[13]['data'])),
                        columns=["PLAYER", "team", "stat", 
                          "dk_line", "dk_oOdds", "dk_uOdds",
                          "fd_line", "fd_oOdds", "fd_uOdds",
                          "m_line", "m_oOdds", "m_uOdds",
                          "pb_line", "pb_oOdds", "pb_uOdds"])

        else:
            # combine all the data for the other stat lines into the main one
            temp = pd.DataFrame(list(zip(PLAYER, team, stat, 
                                 books[2]['data'], books[3]['data'], books[4]['data'], 
                                 books[5]['data'], books[6]['data'], books[7]['data'],
                                 books[8]['data'], books[9]['data'], books[10]['data'],
                                 books[11]['data'], books[12]['data'], books[13]['data'])),
                                columns=["PLAYER", "team", "stat", 
                                  "dk_line", "dk_oOdds", "dk_uOdds",
                                  "fd_line", "fd_oOdds", "fd_uOdds",
                                  "m_line", "m_oOdds", "m_uOdds",
                                  "pb_line", "pb_oOdds", "pb_uOdds"])
            
            data = pd.concat([data, temp])

    # using dk line as the default and averaging the other options when DK is not available
    data["line"] = np.where(data["dk_line"].isna(), 
                           data[["fd_line","m_line","pb_line"]].mean(axis=1, skipna=True),
                           data["dk_line"])
    data["oOdds"] = np.where(data["dk_oOdds"].isna(), 
                           data[["fd_oOdds","m_oOdds","pb_oOdds"]].mean(axis=1, skipna=True),
                           data["dk_oOdds"])
    data["uOdds"] = np.where(data["dk_uOdds"].isna(), 
                           data[["fd_uOdds","m_uOdds","pb_uOdds"]].mean(axis=1, skipna=True),
                           data["dk_uOdds"])

    data = data[["PLAYER", "team", "stat", "line", "oOdds", "uOdds"]]
    # adding the date of the odds
    data.loc[:,'date'] = date.today()  

    driver.close()

    # flattening all the stat lines for each player into a single line instead of multiple entries for each player
    data = data.pivot(index=["PLAYER", "team", "date"],
                columns=["stat"],
                values=["line", "oOdds","uOdds"]
    ).reset_index()
    # combining the multi-column index names into a single name
    data.columns   = ['_'.join(col) for col in data.columns.values]
    
    date_string = str(date.today())
    data.to_csv("../output/{}_odds.csv".format(date_string), index=False)
except Exception as e:
    driver.close()
    print(e)